In [10]:
import numpy as np
import scipy
from scipy import stats
import matplotlib.pyplot as plt
import scipy.optimize as opt
import math
from scipy.special import gamma
import pandas as pd
from scipy.integrate import quad
import numpy.linalg as lin
from scipy.stats import norm

#Readin
data = pd.read_csv("MacroSeries.txt", header = None)
data.columns = ['c','k','w','r','y']

macdata  =  pd.read_csv('MacroSeries.txt', header=None)
macdata.columns = ['ct', 'kt', 'wt', 'rt', 'yt']
                          
def get_z(w, k, alpha): 
    '''Get me a series of z's for some series of k and w and a parameter alpha'''
    zt = np.log(w/((1-alpha)*k**alpha))
    return zt
    
def log_lik_BM_1(w, k, alpha, mu, sigma, rho):
    '''Get me the log likelihood of the z values pdf for some data and params'''
    zvals = get_z(w, k, alpha)
    log_lik_val = 0
    for i in range(len(zvals)):
        if i == 0:
            pdf_val = norm.pdf(mu, loc=mu, scale=sigma)
        else:
            pdf_val = norm.pdf(zvals[i], loc=(rho*zvals[i-1] + (1-rho)*mu), scale=sigma)
        ln_pdf_val = np.log(pdf_val)
        log_lik_val += ln_pdf_val
    return log_lik_val

def crit_BM_1(params, *args):
    '''Define a criterion function to minimise
    in this case the negative of the log likelihood'''
    alpha, mu, sigma, rho = params
    w, k = args
    log_lik_val = log_lik_BM_1(w, k, alpha, mu, sigma, rho)
    neg_log_lik_val = -log_lik_val
    return -log_lik_val

params_init_BM_1 = np.array([0.4, 0.4, 0.4, 0.8])
mle_BM_args = (macdata.wt, macdata.kt)
results_BM_1 = opt.minimize(crit_BM_1, params_init_BM_1, args=mle_BM_args, method ='L-BFGS-B', bounds=((1e-2, 0.99),\
                                                                                                       (1e-2, None),\
                                                                                                (1e-2, None),\
                                                                                                      (-0.99, 0.99)))
alpha_mle, mu_mle, sigma_mle, rho_mle = results_BM_1.x
loglikval_BM = -results_BM_1.fun
IHVCV = results_BM_1.hess_inv.todense()
#Use rick's algorithm to take negative elements of off-diagonal
print('alpha_mle=', alpha_mle, '\n mu_mle=', mu_mle, '\n sigma_mle:', sigma_mle, '\n rho_mle:', rho_mle, '\n Loglikvalmax=', loglikval_BM, "\n InvHessVarCovar = \n", IHVCV)

alpha_mle= 0.5534077576357248 
 mu_mle= 8.106499434665157 
 sigma_mle: 0.09278026792905704 
 rho_mle: 0.8246485429070475 
 Loglikvalmax= 95.85827569290834 
 InvHessVarCovar = 
 [[ 2.94783896e+00 -3.91105055e+01 -2.20282338e-01 -8.61238189e-01]
 [-3.91105055e+01  5.18902999e+02  2.92239258e+00  1.14268028e+01]
 [-2.20282338e-01  2.92239258e+00  1.66392463e-02  6.39042748e-02]
 [-8.61238189e-01  1.14268028e+01  6.39042748e-02  2.56555043e-01]]
